# CS109A Project Group 21
# Baseline Model

*****

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from collections import Counter

### Importing the relevant datasets

First we import two subset of 100 playlists, each subset of 100 playlists to be used for the training and test sets respectively. 

These playlists are randomly chosen from the full one million playlist dataset that has been matched with the Spotify API data, thus including data on track-specific features such as loudness, speechiness and tempo.

In [14]:
train_df = pd.read_csv("data/subset100playlists.csv")
test_df = pd.read_csv("data/subset100playlists_test.csv")

In [15]:
train_df.head()

,Unnamed: 0,index,pid,pos,track_uri,artist_uri_x,track_name,artist_name,artist_uri_y,album_name,...,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,valence
0,0,107,6011,0,spotify:track:2FBUoWkIuQXwayw2RNo5l6,spotify:artist:3TVXtAsR1Inumwj472S9r4,Hotline Bling,Drake,spotify:artist:3TVXtAsR1Inumwj472S9r4,Views,...,2,0.0504,-8.094,1,0.0587,134.960,4,https://api.spotify.com/v1/tracks/2FBUoWkIuQXw...,audio_features,0.539
1,1,107,6011,1,spotify:track:25khomWgBVamSdKw7hzm3l,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,The Hills,The Weeknd,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,Beauty Behind The Madness,...,0,0.1370,-7.011,0,0.0798,135.553,5,https://api.spotify.com/v1/tracks/25khomWgBVam...,audio_features,0.109
2,2,107,6011,2,spotify:track:2NVt7fxr5GsqTkGwYXcNTE,spotify:artist:5lHRUCqkQZCIWeX7xG4sYT,"Flex (Ooh, Ooh, Ooh)",Rich Homie Quan,spotify:artist:5lHRUCqkQZCIWeX7xG4sYT,"Flex (Ooh, Ooh, Ooh)",...,6,0.3410,-7.534,0,0.0496,163.993,4,https://api.spotify.com/v1/tracks/2NVt7fxr5Gsq...,audio_features,0.883
3,3,107,6011,3,spotify:track:5NQbUaeTEOGdD6hHcre0dZ,spotify:artist:6PXS4YHDkKvl1wkIl4V8DL,679 (feat. Remy Boyz),Fetty Wap,spotify:artist:6PXS4YHDkKvl1wkIl4V8DL,679 (feat. Remy Boyz),...,7,0.6250,-5.738,1,0.3180,190.050,4,https://api.spotify.com/v1/tracks/5NQbUaeTEOGd...,audio_features,0.603
4,4,107,6011,4,spotify:track:5hJFhO9dvhJoDvUZZ9iWSw,spotify:artist:360IAlyVv4PCEVjgyMZrxK,Adorn,Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,Kaleidoscope Dream (Deluxe Version),...,11,0.1870,-5.693,0,0.1750,179.063,4,https://api.spotify.com/v1/tracks/5hJFhO9dvhJo...,audio_features,0.235


### Creating the train and test set

We then create the train and test set by keeping select numeric features and the track URI, as a way to maintain a unique ID for each song.

Specifically, for the test set, we split it into two subsets: calibration (20%) and withheld songs (80%), where the calibration songs are used as inputs to the model to predict songs. These predicted songs are then compared to the withheld songs as a comparison metric.

In [18]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_cleaned= train_df.select_dtypes(include=numerics)

var_drop = ["index","pid","pos", "count", "Unnamed: 0"]
df_cleaned = df_cleaned.drop(var_drop, axis =1)

train = pd.concat([df_cleaned, train_df['track_uri']],axis=1)

In [19]:
test_df_cleaned = test_df.select_dtypes(include=numerics)

test_var_drop = ["index","pos", "count", "Unnamed: 0"]
test_df_cleaned = test_df_cleaned.drop(test_var_drop, axis =1)

test_df_cleaned = pd.concat([test_df_cleaned, test_df['track_uri']],axis=1)
calibration, withheld = train_test_split(test_df_cleaned, test_size=0.8, random_state=209, stratify = test_df_cleaned['pid'])

In [20]:
train.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000,13562.000000
mean,0.157796,0.651324,234039.366023,0.685388,0.012731,5.128300,0.186206,-6.434418,0.647028,0.110930,122.022105,3.978985,0.553528
std,0.198003,0.148380,51493.366351,0.167506,0.076586,3.668483,0.144203,2.653691,0.477912,0.109549,28.255147,0.260324,0.226203
min,0.000006,0.144000,19133.000000,0.027900,0.000000,0.000000,0.015700,-34.475000,0.000000,0.022500,34.717000,0.000000,0.034000
25%,0.020000,0.549000,202723.250000,0.574000,0.000000,1.000000,0.092000,-7.703000,0.000000,0.038500,99.973250,4.000000,0.382000
50%,0.073450,0.658000,225991.500000,0.702000,0.000000,5.000000,0.128000,-5.907000,1.000000,0.060500,120.076000,4.000000,0.559000
75%,0.222000,0.757000,256703.500000,0.813000,0.000048,8.000000,0.249000,-4.574750,1.000000,0.141000,140.000000,4.000000,0.737750
max,0.987000,0.980000,826813.000000,0.996000,0.987000,11.000000,1.000000,-0.698000,1.000000,0.956000,210.750000,5.000000,0.990000


### Dropping duplicates
In the baseline model, given that we are only looking at predicting songs based on their similarity to other tracks, we can drop duplicates of the same song from the training set. We do so below.

In [21]:
train.drop_duplicates(subset ="track_uri",keep = False, inplace = True) 
train.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,track_uri
5,0.00563,0.800,159693,0.677,0.00409,0,0.0838,-4.023,1,0.0393,125.004,4,0.547,spotify:track:5bcfWAHLoMfGBdbeyoMaH3
12,0.02940,0.758,222180,0.756,0.00000,10,0.0906,-5.059,0,0.1380,145.068,4,0.406,spotify:track:1rZah1UGXvgiQCqLVlq4hS
19,0.19600,0.855,238530,0.522,0.00000,3,0.1000,-5.540,1,0.2260,139.917,4,0.355,spotify:track:2okC0OWtgS86P2LC18IzHy
22,0.10700,0.703,214757,0.806,0.00000,2,0.1340,-6.415,1,0.1530,94.979,4,0.279,spotify:track:2FdQ4hkbqZ1X930oxxgZZy
26,0.57700,0.670,130293,0.724,0.00000,11,0.7890,-4.141,1,0.3610,145.105,4,0.285,spotify:track:1Wsbr1d2BouNGk2q92mIj7


In [22]:
train = train.set_index('track_uri') 

In [23]:
train.shape

(3098, 13)

### Fitting the training set to a Nearest Neighbors model

This leaves us with 3098 songs in our training set, which we then fit onto an unsupervised Nearest Neighbors learning model, where n_neighbors = 20 and cosine distance is used as a distance of measure. 

Here, we have set the neighbors search algorithm to auto, so that the algorithm attempts to determine the best approach from the training data across the options of BallTree, KDTree, and a brute-force algorithm.

In [25]:
# pivot ratings into song features
model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=20, n_jobs=-1)# fit the dataset
model_knn.fit(train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=20, p=2, radius=1.0)

### Creating a function to make recommendations

We then proceed to create a function that will take as inputs 1) the Nearest Neighbors model fitted on the training data, 2) a playlist id (to identify the songs in the test set), 3) the number of neighbors to select from and 4) number of recommended songs to predict.

This function will return a list of recommended songs.

In [26]:
def make_recommendation(model_knn, playlist_id, n_neighbors, n_recommendations):
    
    calibration_songs = calibration[calibration['pid']==playlist_id]
    calibration_songs_clean = calibration_songs.drop(columns=['pid', 'track_uri'])

    song_freq_dict = {}

    for index, song in calibration_songs_clean.iterrows():
        distances, indices = model_knn.kneighbors(song.values.reshape(1,-1), n_neighbors=n_neighbors)

        for index in indices[0]:
            if song_freq_dict.get(index) is None:
                song_freq_dict[index] = 1
            else:
                song_freq_dict[index] += 1

    k = Counter(song_freq_dict) 

    # Finding n highest values 
    top_songs = [i[0] for i in k.most_common(n_recommendations)]    

    rec_songs = train.iloc[top_songs].index

    return rec_songs

### Measuring success

For starters, we will use the R-precision score to measure how successful our song predictions are. Here, we define a function `r_precision`, which computes number of retrieved relevant tracks divided by the number of known relevant tracks (i.e., the number of withheld tracks). It takes as inputs a list of predicted songs and compares it against a list of the actual songs from the given playlist ("withheld songs").

In [27]:
#Function to calculate r_precision (https://recsys-challenge.spotify.com/rules)
#R-precision is the number of retrieved relevant tracks divided by the number of known relevant tracks (i.e., the number of withheld tracks) 

def r_precision(preds, known):
    for i in known:
        if i in preds:
            song = combined_df[combined_df['track_uri']==i]['track_name']
            print(f'{song} appeared in both our predicted playlist and the known list of songs.')
    score = np.sum(known.isin(preds))/known.shape[0]
    return score

### Running the baseline model on test data

We then run the following code to loop through all the unique playlists in the test set to predict songs based on a random subset of 20% of songs from the given test playlist (`calibration`). The predicted songs are then fed into the R-precision score function together with the remaining 80% of songs from the given test playlist (`withheld`).

For testing purposes, we print out the actual track names of the songs being predicted and the withheld songs, so that we can get a qualitative understanding of the songs.

In [29]:
n_neighbors, n_recommendations = 50, 100
r_precision_scores = []

#Loop through all unique playlists in test set to identify predicted songs
for index, pid in enumerate(withheld['pid'].drop_duplicates()): 
    print (index, pid)
    pred_songs = make_recommendation(model_knn, pid, n_neighbors, n_recommendations)
    validation_set = withheld[withheld.pid == pid].track_uri
    print("Predicted songs\n", combined_df[combined_df['track_uri'].isin(pred_songs)]['track_name'])
    print("Known songs\n", test_df[test_df['track_uri'].isin(validation_set)]['track_name'])
    
    rps = r_precision(list(pred_songs), validation_set)
    r_precision_scores.append(rps)
    
    print(f'Playlist {pid}: The R precision score is {rps}')

0 198457
Predicted songs
 66                              Wat U Mean (Aye, Aye, Aye)
234                                          No Letting Go
300                                              R U Mine?
350                                             Obstacle 1
677                                                   Donk
838                                          Body Language
842                                         Si Tú No Estás
1045                             Bright Lights Bigger City
1267                                Let It Go - feat. Akon
1340     Feeling Right (Everything Is Nice) [feat. Popc...
1384                                         Broken Arrows
1405                      She Doesn't Mind - Pitbull Remix
1417                       Send It (feat. Rich Homie Quan)
1461                                      One In A Million
1713                                      Let Me Down Easy
1740                                       Til My Last Day
1866                          

Predicted songs
 214                                                Ice Box
234                                          No Letting Go
485            Chickenhead (feat. La Chat & Three 6 Mafia)
763                                                 Hollow
791                                           Pretty Hurts
1232                                              No Limit
1375                        The Nights - Felix Jaehn Remix
1551                    Strut - From "The Cheetah Girls 2"
1792                                     That Should Be Me
1900                You Can't Stop Me (feat. Thomas Rhett)
2157                                                  Love
2159                                   Somewhere In My Car
2517                                              Everlong
2869                                            Break Free
2878                                            Ugly Heart
3103             Should I Stay or Should I Go - Remastered
3418                                   

Predicted songs
 76                                              Came To Do
144                               Primetime (feat. Miguel)
300                                              R U Mine?
339                                    Steady, As She Goes
692                                 Wop (Official Version)
838                                          Body Language
1251     Habits (Stay High) - The Chainsmokers Extended...
1320                                     Should’ve Been Us
1321                                    Once In A Lifetime
1342                               El Perdón - Forgiveness
1387                                                  Zimt
1392                                Magnets - A-Trak Remix
1531                                         Dance With Me
1674                                                  Amen
1691                                                   Fly
1740                                       Til My Last Day
1877                                   

Predicted songs
 145                         LoveHate Thing - feat. Sam Dew
334                                             Blue Jeans
480                                            Like A Pimp
517                                                  Whoa!
675                                         Feelin' Myself
910      Let's Go (feat. Big D & Twista) - explicit alb...
1232                                              No Limit
1493                                           Stacy's Mom
1807                                              Been You
1900                You Can't Stop Me (feat. Thomas Rhett)
2621                                    Fell On Black Days
2829                                      Waiting For Love
3187                                             Nightrain
3718                                               Tell Em
4015                                          Last Request
4034              Trap Paris (feat. Quavo & Ty Dolla $ign)
4064                              Frien

Predicted songs
 88                      You Don't Own Me - Candyland Remix
334                                             Blue Jeans
492                                   Sippin On Some Syrup
761                                                Perfect
1056                               Stanky Legg - Main Edit
1089                                           Trap N*ggas
1209                                  Ridin' dirty (Remix)
1222                          We Be Burnin' (Recognize It)
1322                                     Let's Do It Again
1406     This One's For You (feat. Zara Larsson) - Offi...
1454                                                 90210
1692                              Ladies Love Country Boys
1713                                      Let Me Down Easy
1839                              Every Rose Has Its Thorn
1906                                 Tell Me Where To Park
1981                                          Time Is Love
2074                                   

...

Predicted songs
 137                                             Love Faces
260                                             I Want Her
791                                           Pretty Hurts
948                                          Calabria 2008
1084                                            Basketball
1232                                              No Limit
1424                                                 Alarm
1507                                G.N.O. Girls Night Out
1559     Only You-Bad Boy Remix (feat. The Notorious B....
1596                                                Mayday
1719               Sober Saturday Night (feat. Vince Gill)
1770                                      Used To Love You
1807                                              Been You
1866                                                 Human
1900                You Can't Stop Me (feat. Thomas Rhett)
2157                                                  Love
2159                                   

Predicted songs
 22                                               Big Tymin
82                                      Look Alive - Remix
109                                           Arch & Point
343                                        I Melt With You
578                                 Dirt Off Your Shoulder
842                                         Si Tú No Estás
871                                            Rock Bottom
921                                                Tuesday
1417                       Send It (feat. Rich Homie Quan)
1993                                                 Gonna
2035                                       She Get Me High
2198                                             Strangers
2320                                      Live And Let Die
2747                                         Wobble Wobble
2998                                             The Metal
3295                                              The Ride
3330                      Earned It (Fi

Predicted songs
 12                                      King Of Everything
98                            Pull Up (feat. Lil Uzi Vert)
165                                 Neighbors Know My Name
485            Chickenhead (feat. La Chat & Three 6 Mafia)
495                    Int'l Players Anthem (I Choose You)
716                                              Got Money
738                       Home - Fedde Le Grand Radio Edit
1045                             Bright Lights Bigger City
1104                                              Research
1273                      Church - Main Version - Explicit
1288                Five More Hours - Deorro x Chris Brown
1302                                  Heroes (we could be)
1375                        The Nights - Felix Jaehn Remix
1416                                     Perfect Strangers
1482               Dirt Off Your Shoulder / Lying From You
1551                    Strut - From "The Cheetah Girls 2"
1695                                   

Predicted songs
 19                          Bake Sale (feat. Travis Scott)
150                                            Rich Niggaz
334                                             Blue Jeans
530                                  Thuggish Ruggish Bone
713                            Aaron's Party (Come Get It)
795          Beautiful Liar - Main Version / Album Version
825                                            Don't Worry
881                    Mind Your Manners (feat. Icona Pop)
1194                                             White Tee
1285     Crazy Youngsters - From "Pitch Perfect 2" Soun...
1383                                         Pure Grinding
1387                                                  Zimt
1404                                 Heatwave (feat. Akon)
1418                                       Rest Of My Life
1551                    Strut - From "The Cheetah Girls 2"
1674                                                  Amen
1688                                   

Predicted songs
 148                                        Fantasy
155                                        So Gone
487                              Left, Right, Left
518                              Scenario - LP Mix
539                                  Ms. Fat Booty
592                             Often - Kygo Remix
1220                                Duffle Bag Boy
1249               Waves - Robin Schulz Radio Edit
1309                                      Coastin'
1322                             Let's Do It Again
1358                                       Save Me
1708                           Real Men Love Jesus
2055                                         Banjo
2058                                    We Were Us
2128                                        Voices
2310     Learning To Fly - 2011 Remastered Version
2326                   Over The Hills And Far Away
2362                      The Great Gig in the Sky
2430                      Little Lies - Remastered
2455          

### Measuring success

To measure success for the baseline model, we compute the mean of the R-precision scores across all the playlists in the test set. This returns us a value of 0.005, which implies that on average, out of all the withheld songs in the test playlists, we only manage to predict 0.8% of them using our model. This is low -  as a baseline model, it gives us a sense of the challenges that lie ahead in our predictions. 

In [45]:
avg_rp = np.mean(r_precision_scores)

print('Avg. R-Precision Score: ', avg_rp)

Avg. R-Precision Score:  0.008311438433374943


### Challenges and next steps

One of the significant challenges that we have identified could be the fact that the withheld songs in the test data set are not in our original training set. Hence, logically, the model trained on songs in the training set would not be able to return predicted songs that would match 1-to-1 with the withheld songs.

To validate this hypothesis, we ran a quick check on the intersection of songs between both sets, and as expected, only 1552 songs in the withheld list of songs, out of a total of 11074 songs, appear in the training set.

Going ahead, this will mean possibly expanding the training set to increase the overlap in songs between the two sets.

In [47]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 
  
print (len(intersection(list(withheld.track_uri), list(train.index))))
print (len(list(withheld.track_uri)))

1552
11074
